In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from livelossplot import PlotLosses
from keras import backend as tf
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.layers.wrappers import TimeDistributed
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from IPython import display as dp
import os
import pickle

sns.set_style('whitegrid')
sns.despine()

dp.set_matplotlib_formats('retina')

%matplotlib inline

/Users/nickstulov/.virtualenvs/venv3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


A stateful LSTM, reads `horizon` time steps, predicts next one for each, outputs all answers.

In [ ]:
class DataLoader():
    """
    Batches generator.
    
    Takes a directory `path`, sequentially reads `batch_size` files, then
        sequentially reads `horizon` timestamps from each file.
        
    Args:
        path (str): Path to collection.
        batch_size (int): Number of documents per batch.
        horizon (int): Number of timestamps per batch.
        dimension (int): Series vector length.
    """
    def __init__(self, path, batch_size, horizon, dimension):
        self._batch_size = batch_size
        self._horizon = horizon
        self._dimension = dimension
        self._collection = [os.path.join(path, x) for x in os.listdir(path) if x != '.DS_Store']
        self._collection_size = len(self._collection)
        self._num_batches = self._collection_size // self._batch_size + 1
        self._cursor_collection = 0
        self._cursor_file = 0 jk
        with open('sc.pickle', 'rb') as f:
            self._sc = pickle.load(f)
        self._read_next_files()
        self._file_size = 125023
        self._last_batch = self._read_from_files()
        
    def _read_next_files(self):
        """
        Reads the collection by chunks to save memory.
        """
        files_to_read = self._collection[self._cursor_collection : self._cursor_collection + self._batch_size]
        self._cursor_collection += self._batch_size
        if self._cursor_collection > self._collection_size:
            diff = self._cursor_collection % self._collection_size
            self._cursor_collection = diff
            files_to_read.extend(self._collection[0 : diff])
        self._files = [self._sc.transform(pd.read_csv(fpath).drop(['t'], axis=1).values) for fpath in files_to_read]
        
    def _read_from_files(self):
        """
        Reads one timestamp for multiple series.
        
        Returns:
            batch (numpy array): Array of shape (batch_size, 1, dimension).
            reset (bool): Whether states should be reset.
        """
        batch = np.zeros((self._batch_size, self._dimension))
        for i, doc in enumerate(self._files):
            batch[i] = doc[self._cursor_file]
        self._cursor_file += 1
        if self._cursor_file == self._file_size:
            self._cursor_file = self._cursor_file % self._file_size
            self._read_next_files()
            return batch, True
        else:
            return batch, False
        
    def next(self):
        """
        Reads multiple timestamps for multiple series, i.e. complete batch.
        """
        batch = np.zeros((self._batch_size, self._horizon + 1, self._dimension))
        if self._last_batch[1]:
             self._last_batch = self._read_from_files()
        batch[:, 0, :] = self._last_batch[0]
        for i in range(1, self._horizon + 1):
            b, reset = self._read_from_files()
            batch[:, i, :] = b
            if reset:
                self._last_batch = self._last_batch[0], True
                return batch, True
        self._last_batch = batch[:, -1, :], False
        return batch, False

In [ ]:
epochs = 10000
batch_size = 100
num_hidden = 100
num_vars = 4
horizon = 30

In [ ]:
# encoder-decoder

In [ ]:
inputs = Input(batch_shape=(batch_size, horizon, num_vars))
logits = LSTM(num_hidden, stateful=True, return_sequences=True, kernel_regularizer='l2',
              recurrent_regularizer='l2', bias_regularizer='l2', activity_regularizer='l2',
              recurrent_dropout=0.1)(inputs)
outputs = TimeDistributed(Dense(num_vars))(logits)
model = Model(inputs, outputs)

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
dl = DataLoader('data', batch_size, horizon, num_vars)

In [ ]:
liveplot = PlotLosses(figsize=(20,10))
for epoch in range(int(1e4)):
    batch, reset = dl.next()
    hist = model.train_on_batch(batch[:, :-1, :], batch[:, 1:, :])
    if reset:
        model.reset_states()
    liveplot.update({
        'log_mse': np.log(hist)
    })
    liveplot.draw()

In [ ]:
liveplot = PlotLosses(figsize=(20,10))
for epoch in range(int(5e4)):
    batch, reset = dl.next()
    hist = model.train_on_batch(batch[:, :-1, :], batch[:, 1:, :])
    if reset:
        model.layers[1].reset_states()
    liveplot.update({
        'mse': hist
    })
    liveplot.draw()

In [ ]:
vals = pd.read_csv('data_val/data_80.csv').drop(['t'], axis=1).values

In [ ]:
with open('sc.pickle', 'rb') as f:
    sc = pickle.load(f)

In [ ]:
model.reset_states()

In [ ]:
vals = sc.transform(vals)

In [ ]:
vdiffs = np.array([vals[i-1, 0] - vals[i, 0] for i in range(1, len(vals))])

In [ ]:
ticks = np.where(np.abs(vdiffs) > np.mean(np.abs(vdiffs)) + 2 * np.std(np.abs(vdiffs)))[0]

In [ ]:
ticks_on_off = [(ticks[i], ticks[i+1]) for i in range(len(ticks)) if i % 2 == 0]

In [ ]:
ticks_on, ticks_off = tuple(np.array(ticks_on_off).T.tolist())

In [ ]:
from tqdm import tnrange

In [ ]:
# подать 60, первые 30 это ничего, дальше по одному шажку получить state для сдвига на один
# цель: иметь state для k * horizon + i шага \forall i \in [0, horizon)
# для этого нужно иметь предысторию размером horizon и сохраненный state в позиции (k-1) * horizon + i

In [ ]:
model.reset_states()
reset_states = model.layers[1].states

In [ ]:
predicted = [*vals[:horizon]]
states = [reset_states for _ in range(horizon)]
disturbance = False
for i in tnrange(horizon, len(vals) // horizon * horizon, horizon):
    for j in range(horizon):
        inputs = np.zeros((batch_size, horizon, num_vars))
        state = states[i-horizon+j]
        model.layers[1].states[0].assign(state[0].value())
        model.layers[1].states[1].assign(state[1].value())
        if i+j in ticks_on:
            disturbance = True
        elif i+j in ticks_off:
            disturbance = False
        if disturbance:
            inputs[0, :-1] = predicted[i-horizon+j:i+j-1]
            inputs[0, -1] = vals[i+j]
        else:
            inputs[0] = predicted[i-horizon+j:i+j]
        pred = model.predict_on_batch(inputs)[0][-1]
        predicted.append(pred)
        states.append(model.layers[1].states)

In [ ]:
predicted = np.array(predicted)

In [ ]:
concat_pred = np.array(sc.inverse_transform(predicted))

In [ ]:
concat_true = sc.inverse_transform(vals)[:150030]

In [ ]:
print(concat_pred.shape, concat_true.shape)

In [ ]:
var_names = ['$V$', '$\\varphi$', '$P$', '$Q$']

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20,20))
fig.tight_layout(h_pad=4.0)
for i, (ax, name) in enumerate(zip(axes.flatten(), var_names)):
    ax.plot(concat_pred[:, i], c='r', label='pred')
    ax.plot(concat_true[:, i], c='b', label='true')
    ax.set_title(name, fontsize='xx-large')
    ax.legend(loc='best', fontsize='xx-large')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20,20))
fig.tight_layout(h_pad=4.0)
for i, (ax, name) in enumerate(zip(axes.flatten(), var_names)):
    ax.plot(concat_pred_new[:, i], c='r', label='pred')
    ax.plot(concat_true[:, i], c='b', label='true')
    ax.set_title(name, fontsize='xx-large')
    ax.legend(loc='best', fontsize='xx-large')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20,20))
fig.tight_layout(h_pad=4.0)
for i, (ax, name) in enumerate(zip(axes.flatten(), var_names)):
    ax.plot(((np.repeat(concat_true[0, i], ???) - concat_true[:, i]) / concat_true[:, i]) ** 2, 'g', label='pers')
    ax.plot(((concat_pred[:, i] - concat_true[:, i]) / concat_true[:, i]) ** 2, 'r', label='pred')
    ax.set_title(name, fontsize='xx-large')
    ax.legend(loc='best', fontsize='xx-large')

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(np.mean(((concat_true[:-1] - concat_true[1:]) / concat_true[1:]) ** 2, axis=1), 'g', label='pers')
plt.plot(np.mean(((concat_pred - concat_true) / concat_true) ** 2, axis=1), 'r', label='pred')

In [ ]:
model.save('lstm_scaled_50k.h5')

In [2]:
from keras.models import load_model

In [4]:
def nrmse(y_true, y_pred):
    return K.mean(K.sqrt(K.mean(K.sum((y_true - y_pred) ** 2, axis=2), axis=1)) \
                  / K.sqrt(K.mean(K.sum(y_true ** 2, axis=2), axis=1)))

In [7]:
import keras.backend as K

In [8]:
m = load_model('lstm_supervised_scaled2_50k.h5', custom_objects={'nrmse': nrmse})

In [9]:
from keras.utils import plot_model

In [11]:
plot_model(m, show_layer_names=False, rankdir='LR')